In [10]:
import boto3
import sagemaker
from sagemaker import get_execution_role
# from sagemaker.image_uris import retrieve
from sagemaker.s3 import *
import sys

if int(sagemaker.__version__.split('.')[0]) == 2:
    !{sys.executable} -m pip install sagemaker==1.72.0
    print("Installing previous SageMaker Version. Please restart the kernel")
else:
    print("Version is good")

role = get_execution_role()
sess = sagemaker.Session(default_bucket=None)
region = boto3.session.Session().region_name
print("Region = {}".format(region))

sm = boto3.Session().client('sagemaker')


Version is good
Region = eu-central-1


In [11]:

# see https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html for which inputs to use
# see https://github.com/aws/deep-learning-containers/blob/master/available_images.md for registry paths with custom algorithms

PREPROCESSING_IMAGE = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04"
PREPROCESS_INSTANCE = "ml.m5.xlarge"

TRAINING_IMAGE = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04"
TRAINING_INSTNCE = "ml.g4dn.xlarge" 

print ("Preprocessing image: ", PREPROCESSING_IMAGE)
print ("Training image: ", TRAINING_IMAGE)



Preprocessing image:  763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04
Training image:  763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04


In [12]:
!pip install sagemaker-experiments 
from sagemaker.analytics import ExperimentAnalytics
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [16]:
sagemaker_local_bucket = sess.default_bucket() # Alternatively you can use our custom bucket here. 
original_data_bucket = "treetracker-training-images"
prefix = 'sagemaker-modelmonitor' # use this prefix to store all files pertaining to this workshop.

dataprefix = prefix + '/data'
traindataprefix = prefix + '/train_data'
testdataprefix = prefix + '/test_data'
testdatanolabelprefix = prefix + '/test_data_no_label'
trainheaderprefix = prefix + '/train_headers'

dataset_key = "imnet" # use this to restrict to a particular directory
train_key = "train"
validation_key = "validation"
test_key = "test"
s3_raw = 's3://{}/{}/'.format(original_data_bucket, dataset_key)
sagemaker_train = 's3://{}/{}/'.format(sagemaker_local_bucket, train_key)
sagemaker_validation = 's3://{}/{}/'.format(sagemaker_local_bucket, validation_key)
sagemaker_test = 's3://{}/{}/'.format(sagemaker_local_bucket, test_key)


In [17]:
train_data_location = original_data_bucket + '/' + traindataprefix
test_data_location = original_data_bucket +'/'+testdataprefix
print("Training data location = {}".format(train_data_location))
print("Test data location = {}".format(test_data_location))

Training data location = treetracker-training-images/sagemaker-modelmonitor/train_data
Test data location = treetracker-training-images/sagemaker-modelmonitor/test_data


In [18]:
# TODO: Figure out preprocessing instance jobs
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor

pre_input = ProcessingInput(source=s3_raw, destination=None, input_name="pre")
pre_output = [ProcessingOutput(source=None, destination=sagemaker_train),
              ProcessingOutput(source=None, destination=sagemaker_validation),
              ProcessingOutput(source=None, destination=sagemaker_test)]
            
              
script_processor = ScriptProcessor(image_uri=PREPROCESSING_IMAGE,
                role=role,
                instance_count=1,
                instance_type=PREPROCESS_INSTANCE)

preprocessing_script = None # Put path to preprocessing script here

TypeError: __init__() missing 1 required positional argument: 'command'

In [ ]:
script_processor.run(preprocessing_script, 
                    inputs=pre_input,
                    outputs=pre_output)

In [6]:
from sagemaker.pytorch import PyTorch

# This is where you can add hyperparameters, framework used, point to the script, and define instances you want to train on. 
# ALl of this information is represented as environment variables passed to the instance. In your script, you can refer to these variables or 
# the argument. 
estimator = PyTorch(entry_point='mobilenet_v2.py',
                    role=role,
                    framework_version='1.7.0',
                    train_instance_count=1,
                    train_instance_type=TRAINING_INSTANCE,
                    hyperparameters={
                        'epochs': 5,
                        'backend': 'gloo',
                        'train_split': 0.7, 
                        'log_interval': 200
                    },
                   
                   )

ImportError: cannot import name 'empty_framework_version_warning'

In [7]:
estimator.fit({"training": s3_train})

NameError: name 'estimator' is not defined